# 02 WACC & DCF Sanity Checks
This notebook computes the Weighted Average Cost of Capital (WACC), performs Discounted Cash Flow (DCF) sanity checks, and validates assumptions.

In [ ]:
import pandas as pd
import numpy as np
import math

## Input Assumptions

In [ ]:
# Capital structure
equity_value = 1000  # market cap
debt_value = 400    # total debt
cash = 100

# Costs
cost_of_equity = 0.10   # CAPM
cost_of_debt = 0.05     # pre-tax
tax_rate = 0.25

## Compute WACC

In [ ]:
V = equity_value + debt_value - cash
E = equity_value
D = debt_value

wacc = (E/V)*cost_of_equity + (D/V)*cost_of_debt*(1 - tax_rate)
print("WACC:", round(wacc*100,2), "%")

## DCF Sanity Example

In [ ]:
# Forecast Free Cash Flows (FCF)
fcf = [120, 135, 150, 160, 170]
terminal_growth = 0.02

def npv(rate, cashflows):
    return sum(cf/((1+rate)**(i+1)) for i, cf in enumerate(cashflows))

# PV of forecast
pv_fcf = npv(wacc, fcf)

# Terminal value using Gordon Growth
tv = fcf[-1]*(1+terminal_growth)/(wacc-terminal_growth)
pv_tv = tv/((1+wacc)**len(fcf))

enterprise_value = pv_fcf + pv_tv
equity_value_dcf = enterprise_value - D + cash
print("Enterprise Value:", round(enterprise_value,2))
print("Equity Value (DCF):", round(equity_value_dcf,2))

## Sensitivity Analysis

In [ ]:
wacc_range = np.linspace(0.07, 0.13, 5)
growth_range = [0.01, 0.02, 0.03]

sens = {}
for g in growth_range:
    row = []
    for r in wacc_range:
        tv = fcf[-1]*(1+g)/(r-g)
        pv_tv = tv/((1+r)**len(fcf))
        ev = npv(r, fcf) + pv_tv
        eq = ev - D + cash
        row.append(round(eq,2))
    sens[g] = row
pd.DataFrame(sens, index=[f"WACC {round(r*100,1)}%" for r in wacc_range])